## Notebook test for [Ticket](https://iguazio.atlassian.net/browse/ML-6609)

#### Create a project

In [1]:
import mlrun

In [2]:
mlrun.__version__

'1.7.0-rc26'

In [3]:
project = mlrun.get_or_create_project("testing-model-endpoint")

> 2024-06-26 09:14:32,417 [info] Loading project from path: {"path":"./","project_name":"testing-model-endpoint"}
> 2024-06-26 09:14:47,802 [info] Project loaded successfully: {"path":"./","project_name":"testing-model-endpoint","stored_in_db":true}


In [4]:
#Clean up monitoring data
project.disable_model_monitoring(delete_stream_function=True)

> 2024-06-26 09:14:47,968 [info] Model Monitoring is being disable: {"project_name":"testing-model-endpoint"}
> 2024-06-26 09:14:47,969 [info] Model Monitoring disabled: {"project":"testing-model-endpoint"}


In [5]:
project.enable_model_monitoring(wait_for_deployment=True)

2024-06-26 09:14:52  (info) Deploying function
2024-06-26 09:14:52  (info) Building
2024-06-26 09:14:53  (info) Staging files and preparing base images
2024-06-26 09:14:53  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2024-06-26 09:14:53  (info) Building processor image
2024-06-26 09:16:53  (info) Build complete
2024-06-26 09:17:32  (info) Function deploy complete
2024-06-26 09:14:48  (info) Deploying function
2024-06-26 09:14:48  (info) Building
2024-06-26 09:14:48  (info) Staging files and preparing base images
2024-06-26 09:14:48  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2024-06-26 09:14:48  (info) Building processor image
2024-06-26 09:16:47  (info) Build complete
2024-06-26 09:17:31  (info) Function deploy complete
2024-06-26 09:14:50  (info) Deploying function
2024-06-26 09:14:50  (info) Building
2024-06-26 09:14:51  (info) Staging files and preparing base images
2024-06-26

### Train and deploy a model with tag==v1 and label test=1

In [6]:
%%writefile train.py
from sklearn.datasets import make_classification
from mlrun.frameworks.sklearn import apply_mlrun
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
import pandas as pd

def train(context,i):
    X,y = make_classification(n_samples=100,n_features=10,random_state=i)
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=i)
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train,columns=["label"])
    training_set = pd.concat([X_train,y_train],axis=1)
    model = LinearRegression()
    model.fit(X_train,y_train)
    context.log_model(key=f"model",body=pickle.dumps(model),model_file=f'model_{i}.pkl',training_set=training_set,label_column="label",tag=f"v{i}",labels={"test":i})


Overwriting train.py


In [7]:
project.set_function(func='train.py',kind='job',image="mlrun/mlrun",handler='train',name='train')

In [8]:
i=1

In [9]:
run = project.run_function('train',params={"i":i})

> 2024-06-26 09:18:08,425 [info] Storing function: {"db":"http://mlrun-api:8080","name":"train-train","uid":"7010715de3b34cbe8606196d51365bb6"}
> 2024-06-26 09:18:08,729 [info] Job is running in the background, pod: train-train-84jt6
> 2024-06-26 09:18:13,103 [info] To track results use the CLI: {"info_cmd":"mlrun get run 7010715de3b34cbe8606196d51365bb6 -p testing-model-endpoint","logs_cmd":"mlrun logs 7010715de3b34cbe8606196d51365bb6 -p testing-model-endpoint"}
> 2024-06-26 09:18:13,103 [info] Or click for UI: {"ui_url":"https://dashboard.default-tenant.app.dev55.lab.iguazeng.com/mlprojects/testing-model-endpoint/jobs/monitor/7010715de3b34cbe8606196d51365bb6/overview"}
> 2024-06-26 09:18:13,104 [info] Run execution finished: {"name":"train-train","status":"completed"}


project,uid,iter,start,state,kind,name,labels,inputs,parameters,results,artifacts
testing-model-endpoint,...51365bb6,0,Jun 26 09:18:11,completed,run,train-train,v3io_user=normal-userkind=jobowner=normal-usermlrun/client_version=1.7.0-rc26mlrun/client_python_version=3.9.18host=train-train-84jt6,,i=1,,modelmodel


> 2024-06-26 09:18:19,876 [info] Run execution finished: {"name":"train-train","status":"completed"}


In [10]:
serving = mlrun.new_function('serving-model',kind='serving',image='mlrun/mlrun')

In [11]:
graph = serving.set_topology("flow", engine="sync")
router = graph.add_step("*mlrun.serving.ModelRouter", name="model-routing")

In [12]:
models_uri = [model.uri for model in project.list_models(tag=f"v{i}")]
len(models_uri)

1

In [13]:
from tqdm import tqdm
for uri in tqdm(models_uri):
    router.add_route(key=f'model',model_path=uri,class_name='mlrun.frameworks.sklearn.SKLearnModelServer')

100%|██████████| 1/1 [00:00<00:00, 11096.04it/s]


In [14]:
serving.set_tracking()

In [15]:
project.deploy_function(serving)

> 2024-06-26 09:18:19,962 [info] Starting remote function deploy
2024-06-26 09:18:20  (info) Deploying function
2024-06-26 09:18:20  (info) Building
2024-06-26 09:18:20  (info) Staging files and preparing base images
2024-06-26 09:18:20  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2024-06-26 09:18:20  (info) Building processor image
2024-06-26 09:19:45  (info) Build complete
2024-06-26 09:20:31  (info) Function deploy complete
> 2024-06-26 09:20:41,192 [info] Successfully deployed function: {"external_invocation_urls":["testing-model-endpoint-serving-model.default-tenant.app.dev55.lab.iguazeng.com/"],"internal_invocation_urls":["nuclio-testing-model-endpoint-serving-model.default-tenant.svc.cluster.local:8080"]}


DeployStatus(state=ready, outputs={'endpoint': 'http://testing-model-endpoint-serving-model.default-tenant.app.dev55.lab.iguazeng.com/', 'name': 'testing-model-endpoint-serving-model'})

In [16]:
inputs = [[-0.13554955389680073,0.0076506575048474225,0.6287761723991921,-0.8751269647375463,-1.0660002219502747,-1.449969473671631,-1.1047354035566603,0.20042964461755708,0.6400321379372533,0.0785128175305061,-0.16572271960124885,0.8553736325613692,-0.4950274467481983,0.3036076443871318,-0.9177524223010839,0.3994151051172195,-0.2588778273252802,0.7705348945630903,0.14223688502019674,-0.722042747480339][:10]]

In [17]:
serving.invoke(f"/v2/models/model/infer", {"inputs": inputs})

> 2024-06-26 09:20:41,258 [info] Invoking function: {"method":"POST","path":"http://nuclio-testing-model-endpoint-serving-model.default-tenant.svc.cluster.local:8080/v2/models/model/infer"}


{'id': 'b7dbe74e-ded3-4488-955d-368988488ef5',
 'model_name': 'model',
 'outputs': [[0.39645381565027626]],
 'model_version': 'v1'}

In [18]:
db = mlrun.get_run_db()

In [19]:
end_point = db.list_model_endpoints(project=project.name,model=f"model:v{i}")[0]

#### Test model tag and label

In [20]:
#test model endpoint labels
assert end_point.metadata.labels == {'test': i}

In [21]:
#test model endpoint tag
assert end_point.spec.model.split(":")[-1]==f"v{i}"

### Train and deploy a model with tag==v2 and label test=2

In [22]:
%%writefile train.py
from sklearn.datasets import make_classification
from mlrun.frameworks.sklearn import apply_mlrun
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
import pandas as pd

def train(context,i):
    X,y = make_classification(n_samples=100,n_features=10,random_state=i)
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=i)
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train,columns=["label"])
    training_set = pd.concat([X_train,y_train],axis=1)
    model = LinearRegression()
    model.fit(X_train,y_train)
    context.log_model(key=f"model",body=pickle.dumps(model),model_file=f'model_{i}.pkl',training_set=training_set,label_column="label",tag=f"v{i}",labels={"test":i})


Overwriting train.py


In [23]:
project.set_function(func='train.py',kind='job',image="mlrun/mlrun",handler='train',name='train')

In [24]:
i=2

In [25]:
run = project.run_function('train',params={"i":i})

> 2024-06-26 09:20:41,923 [info] Storing function: {"db":"http://mlrun-api:8080","name":"train-train","uid":"5a59123a60e74e739b66197859cbc063"}
> 2024-06-26 09:20:42,189 [info] Job is running in the background, pod: train-train-dt6br
> 2024-06-26 09:20:46,572 [info] To track results use the CLI: {"info_cmd":"mlrun get run 5a59123a60e74e739b66197859cbc063 -p testing-model-endpoint","logs_cmd":"mlrun logs 5a59123a60e74e739b66197859cbc063 -p testing-model-endpoint"}
> 2024-06-26 09:20:46,572 [info] Or click for UI: {"ui_url":"https://dashboard.default-tenant.app.dev55.lab.iguazeng.com/mlprojects/testing-model-endpoint/jobs/monitor/5a59123a60e74e739b66197859cbc063/overview"}
> 2024-06-26 09:20:46,573 [info] Run execution finished: {"name":"train-train","status":"completed"}


project,uid,iter,start,state,kind,name,labels,inputs,parameters,results,artifacts
testing-model-endpoint,...59cbc063,0,Jun 26 09:20:45,completed,run,train-train,v3io_user=normal-userkind=jobowner=normal-usermlrun/client_version=1.7.0-rc26mlrun/client_python_version=3.9.18host=train-train-dt6br,,i=2,,modelmodel


> 2024-06-26 09:20:53,329 [info] Run execution finished: {"name":"train-train","status":"completed"}


In [26]:
serving = mlrun.new_function('serving-model',kind='serving',image='mlrun/mlrun')

In [27]:
graph = serving.set_topology("flow", engine="sync")
router = graph.add_step("*mlrun.serving.ModelRouter", name="model-routing")

In [28]:
models_uri = [model.uri for model in project.list_models(tag=f"v{i}")]
len(models_uri)

1

In [29]:
assert "v2" in models_uri[0]

In [30]:
from tqdm import tqdm
for uri in tqdm(models_uri):
    router.add_route(key=f'model',model_path=uri,class_name='mlrun.frameworks.sklearn.SKLearnModelServer')

100%|██████████| 1/1 [00:00<00:00, 11715.93it/s]


In [31]:
serving.set_tracking()

In [32]:
project.deploy_function(serving)

> 2024-06-26 09:20:53,410 [info] Starting remote function deploy
2024-06-26 09:20:53  (info) Deploying function
2024-06-26 09:20:53  (info) Building
2024-06-26 09:20:53  (info) Staging files and preparing base images
2024-06-26 09:20:53  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2024-06-26 09:20:53  (info) Building processor image
2024-06-26 09:22:09  (info) Build complete
2024-06-26 09:22:17  (info) Function deploy complete
> 2024-06-26 09:22:24,334 [info] Successfully deployed function: {"external_invocation_urls":["testing-model-endpoint-serving-model.default-tenant.app.dev55.lab.iguazeng.com/"],"internal_invocation_urls":["nuclio-testing-model-endpoint-serving-model.default-tenant.svc.cluster.local:8080"]}


DeployStatus(state=ready, outputs={'endpoint': 'http://testing-model-endpoint-serving-model.default-tenant.app.dev55.lab.iguazeng.com/', 'name': 'testing-model-endpoint-serving-model'})

In [33]:
inputs = [[-0.13554955389680073,0.0076506575048474225,0.6287761723991921,-0.8751269647375463,-1.0660002219502747,-1.449969473671631,-1.1047354035566603,0.20042964461755708,0.6400321379372533,0.0785128175305061,-0.16572271960124885,0.8553736325613692,-0.4950274467481983,0.3036076443871318,-0.9177524223010839,0.3994151051172195,-0.2588778273252802,0.7705348945630903,0.14223688502019674,-0.722042747480339][:10]]

In [34]:
serving.invoke(f"/v2/models/model/infer", {"inputs": inputs})

> 2024-06-26 09:22:24,397 [info] Invoking function: {"method":"POST","path":"http://nuclio-testing-model-endpoint-serving-model.default-tenant.svc.cluster.local:8080/v2/models/model/infer"}


{'id': '18857906-1b09-423f-aab9-b6bda56deaec',
 'model_name': 'model',
 'outputs': [[-36716687454529.34]],
 'model_version': 'v2'}

In [35]:
db = mlrun.get_run_db()

In [36]:
end_point = db.list_model_endpoints(project=project.name,model=f"model:v{i}")[0]

#### Test model tag and label

In [37]:
#test model endpoint labels
assert end_point.metadata.labels == {'test': i}

In [38]:
#test model endpoint tags
assert end_point.spec.model.split(":")[-1]==f"v{i}"

In [39]:
db.delete_project(name=project.name,deletion_strategy="cascade")

> 2024-06-26 09:22:24,905 [info] Project is being deleted: {"project_name":"testing-model-endpoint"}
> 2024-06-26 09:22:52,101 [info] Project deleted: {"project_name":"testing-model-endpoint"}
